In [1]:
import pandas as pd
import numpy as np
import seaborn as sns


In [2]:
df=pd.read_csv('../Dataset/Life Expectancy Data.csv')

In [3]:
df.columns=df.columns.str.strip()

In [4]:
df.drop(['thinness 5-9 years', 'Hepatitis B','GDP','Population','Total expenditure'], axis=1,inplace=True)


In [5]:


# Drop rows with missing target variable values
df.dropna(subset=['Life expectancy'], inplace=True)


In [6]:
df['Adult Mortality'].fillna(df['Adult Mortality'].mean(),inplace=True)
df['BMI'].fillna(df['BMI'].mean(),inplace=True)
df['Diphtheria'].fillna(df['Diphtheria'].mean(),inplace=True)
df['Polio'].fillna(df['Polio'].mean(),inplace=True)




In [14]:
df.isnull().sum()

Country                              0
Year                                 0
Status                               0
Life expectancy                      0
Adult Mortality                      0
infant deaths                        0
Alcohol                            193
percentage expenditure               0
Measles                              0
BMI                                  0
under-five deaths                    0
Polio                                0
Diphtheria                           0
HIV/AIDS                             0
thinness  1-19 years                32
Income composition of resources    160
Schooling                          160
dtype: int64

In [7]:
df.columns

Index(['Country', 'Year', 'Status', 'Life expectancy', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Measles', 'BMI',
       'under-five deaths', 'Polio', 'Diphtheria', 'HIV/AIDS',
       'thinness  1-19 years', 'Income composition of resources', 'Schooling'],
      dtype='object')

In [7]:
categorical_columns=[]
numerical_columns=[]
for i in df.columns:
    if df[i].dtypes=='object':
        categorical_columns.append(i)
    else:
        numerical_columns.append(i)
y=df['Life expectancy']        
numerical_columns.remove('Life expectancy')
        
            

In [8]:
#test train split
from sklearn.model_selection import train_test_split

# Define the features and target variable
X = df.loc[:,df.columns!='Life expectancy']


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shape of the training and testing sets
print("Training data shape:", X_train.shape)
print("Testing data shape:", X_test.shape)


Training data shape: (2342, 16)
Testing data shape: (586, 16)


In [9]:
#direct catboost
from catboost import CatBoostRegressor

# Create a CatBoostRegressor model
cat_reg = CatBoostRegressor()

# Train the model on the training set
cat_reg.fit(X_train, y_train)

# Evaluate the model on the testing set
score = cat_reg.score(X_test, y_test)

print("Model score: ", score)


CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="Serbia": Cannot convert 'b'Serbia'' to float

In [9]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Perform one-hot encoding on the Country and Status columns
ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded = ohe.fit_transform(df[categorical_columns])
encoded_cols = ohe.get_feature_names(categorical_columns)
df_encoded = pd.DataFrame(encoded, columns=encoded_cols)

# Perform normalization on the remaining columns
scaler = StandardScaler()
normalized = scaler.fit_transform(df[numerical_columns])
df_normalized = pd.DataFrame(normalized, columns=numerical_columns)

# Concatenate the encoded and normalized columns back into a single DataFrame
df_preprocessed = pd.concat([df_encoded, df_normalized], axis=1)


C:\Users\Utsa Rabbani\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
len(df_preprocessed.columns)

199

In [15]:
df_preprocessed['Life expectancy']=y

In [11]:
#df_preprocessed.to_csv('df_preprocessed.csv', index=False)


In [16]:
df_preprocessed.isnull().columns

Index(['Country_Afghanistan', 'Country_Albania', 'Country_Algeria',
       'Country_Angola', 'Country_Antigua and Barbuda', 'Country_Argentina',
       'Country_Armenia', 'Country_Australia', 'Country_Austria',
       'Country_Azerbaijan',
       ...
       'Measles', 'BMI', 'under-five deaths', 'Polio', 'Diphtheria',
       'HIV/AIDS', 'thinness  1-19 years', 'Income composition of resources',
       'Schooling', 'Life expectancy'],
      dtype='object', length=200)

In [17]:

from sklearn.impute import KNNImputer

# create an instance of the KNNImputer with the desired number of neighbors
imputer = KNNImputer(n_neighbors=2)

# fit the imputer to the data and transform it to fill missing values
df_preprocessed = pd.DataFrame(imputer.fit_transform(df_preprocessed), columns=df_preprocessed.columns)

# df_preprocessed now contains the imputed values and has the same columns and index as the original df dataframe


In [19]:
df_preprocessed.isnull().sum().sum()

0

In [20]:
df_preprocessed.shape

(2928, 200)

In [21]:
df_preprocessed.to_csv('df_preprocessed.csv', index=False)
